# Calculators

### Exceptions
#### Try statements
Exceptions gives the programmer the ability to let the program continue running without terminating by handling the exception (Try statements)

Try statements handle exceptions


In [2]:
try:
    x = 1/0
except ZeroDivisionError as e:
    print('handling a', type(e))
    x = 0

handling a <class 'ZeroDivisionError'>


In [15]:
def invert(x):
    result = 1/x
    print('Never printed if x is 0')
    return result

def safe_invert(x):
    try:
        return invert(x)
    except ZeroDivisionError as e:
        return str(e)
safe_invert(0)

'division by zero'

### Example of using Exception: Reduce

In [24]:
from operator import truediv

def reduce(f, s, initial):
    """Combine elements of s pairwise using f, starting with initial
    >>> reduce(mul, [2, 4, 8], 1) # mul(mul(mul(1, 2), 4), 8)
    64
    """
    for x in s:
        initial = f(initial, x)
    return initial

def reduce_rec(f, s, initial):
    if not s:
        return initial
    else:
        first, rest = s[0], s[1:]
        return reduce_rec(f, rest, f(initial, first))


def divide_all(n, s):
    try:
        return reduce_itr(truediv, s, n)
    except ZeroDivisionError:
        return float('inf')


implementation of reduce doesn't need to know anything about zero division errors, so it's divide_all that handles the error, creating a separation of concerns in our program.

## Programming Languages
a computer typically executes programs written in many different programming Languages

**Machine Languages**: statements are interpreted by the hardware itself
- a fixed set of instructions invoke operations implemented by the circuitry of the central processing unit (CPU)
- operators refer to specific hardware memory addresses; no abstraction mechanisms


**High-level languages**: statements & expressions are interpreted by another program or compiled (translated) into another language
- provide means of abstraction such as naming, function definition, and objects.

### Metalinguistic Abstraction

a powerful form of abstraction is to define a new language that is tailored to a particular type of application or problem domain.


A programming language has:
- **syntax:** the legal statements and expressions in the language
- **semantics:** the execution/evaluation rule for those statements and expressions

to create a new programming language, you either need a:
- **specification:** a document describe the precise syntax and semantics of the language
- Canonical implementation: An interpreter or compiler for the language



### Parsing

To interpret text as a programming language, you first need to parse it into some structure to perform some interpretation.

#### Reading Scheme Lists

A scheme list is written as elements in parentheses:

`(<element n> <element n> ... <element n>)`

each `<element>` can be a combination or primitive

___

A Parser takes text and returns an expression

### Scheme-Syntax Calculator

The value of a calculator expression is defined recursively.

**Primitive:** A number evaluates to itself.

**Call:** a call expression evaluates to its arguments values combined by an operator.
- `+`: Sum of the arguments
- `*`: product of the arguments
- `-`: If one argument, negate it. If more than one, subtract the rest from the first.
- `/`: if one argument, invert it. If more than one, divide the rest from the first.


#### The Eval Function

the `eval` function computes the value of an expression, which is always a number.

It is a generic function that dispatches on the type of the expression (primitive or call).

In [23]:
def calc_eval(exp):
    if type(exp) in (int, float):
        return exp
    elif isinstance(exp, Pair):
        arguments = exp.second.map(calc_eval)
        return calc_apply(exp.first, arguments)
    else:
        raise TypeError

#### Applying Built-in Operators
The `apply` function applies some operation to a (Scheme) list of argument values.

In calculator, all operators are named by built-in operators: `+, -, * /`

In [25]:
def calc_apply(operator, args):
    if operator == '+':
        return reduce(add, args, 0)
    elif operator == '-':
        if len(args) == 0:
            raise TypeError(operator + ' requires at least 1 argument')
        if len(args) == 1:
            return -args.first
        else:
            return reduce(sub, args, 0)
    elif operator == '*':
        return reduce(mul, args, 1)
    elif operator == '/':
        if len(args) == 0:
            raise TypeError(operator + ' requires at least 1 argument')
        elif len(args) == 1:
            return 1 / args.first
        else:
            return reduce(truediv, args.second, args.first)
    else:
        raise TypeError(operator + ' is an unknown operator.')


#### Interactive Interpreter (read-evalutate-print loop)
The user interface for many programming language is an interactive interpreter.
- print a prompt
- **read** text input from the user.
- pase the text into an expression.
- **evaluate** the expression.
- if any errors occur, report those errors, otherwise,
- **print** the value of the expression and repeat

In [27]:
def read_eval_print_loop():
    while True:
        try:
            src = buffer_input()
            while src.more_on_line:
                expression = scheme_read(src)
                print(calc_eval(expression))
        except (SyntaxError, TypeError, ValueError, ZeroDivisionError) as err:
            print(type(err).__name__ + ':', err)
        except (KeyboardInterrupt, EOFError):  # <Control + D>, etc
            print('Calculation completed.')
            return

#### Raising Exceptions

Exceptions are raised within lexical analysis, syntactic analysis, eval, and apply.

**Example exceptions**:
- **lexical analysis**: The token `2.3.44444` raises `ValueError('Invalid numeral')`
- **Syntactic analysis**: An extra `)` raises `SyntaxError('unexpected token')`
- **Eval:** an empty combination raises `TypeError("() is not a number or call expression")`
- **Apply:** no arguments to `-` raises `TypeError("- requires at least 1 argument")`

#### Handling Exceptions
All of the handling of the exceptions happens in one place.

A well-designed interactive interpreter should not halt complete on an error, so that the user has an opportunity to try again in the current environment